In [1]:
import sys
sys.path.append("..")
sys.path.append("../model")
from trajcl import TrajCL
from config import Config

In [2]:
conf = Config()

conf.dataset = 'nyc'
conf.post_value_updates()

In [3]:
model = TrajCL()

/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [4]:
model

TrajCL(
  (clmodel): MoCo(
    (criterion): CrossEntropyLoss()
    (encoder_q): DualSTB(
      (pos_encoder): PositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (structural_attn): TransformerEncoder(
        (layers): ModuleList(
          (0-1): 2 x TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
            )
            (linear1): Linear(in_features=512, out_features=2048, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=2048, out_features=512, bias=True)
            (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (spatial_attn): SpatialMSM(


In [6]:
from ah_databricks_data_loader import DatabricksDataLoader

# Instantiate the DatabricksDataLoader.
ddl = DatabricksDataLoader()

# Load data.
test_df = ddl.load_as_spark(schema="datascience_scratchpad", table="nyc_traj_data_v18")

:: loading settings :: url = jar:file:/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sagemaker-user/.ivy2/cache
The jars for the packages stored in: /home/sagemaker-user/.ivy2/jars
io.delta#delta-sharing-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-179718a3-ae74-4574-8525-70d866752ae5;1.0
	confs: [default]
	found io.delta#delta-sharing-spark_2.12;3.2.0 in central
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found io.delta#delta-sharing-client_2.12;1.0.5 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging;1.2 in central
	found commons-codec#commons-codec;1.11 in central
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-o

In [ ]:
# test_df.count()

In [ ]:
# test_df.show(5)

In [6]:
test_df.write.mode("overwrite").parquet("/home/sagemaker-user/TrajCL/data/nyc/nyc_data_v18")

25/09/08 18:15:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
# test_df.count()

In [5]:
from glob import glob
import pandas as pd
parquet_files = glob("/home/sagemaker-user/TrajCL/data/nyc/nyc_data_v18/*.parquet")
total_count = 0
df_list = []
for file in parquet_files:
    df = pd.read_parquet(file)
    df_list.append(df)
    total_count += len(df)


print(f"Total records in repartitioned files: {total_count}")

Total records in repartitioned files: 1445257


In [6]:
df_list[0].head()

,userid,traj_date,timestamps,wgs_seq,employername,partition_id,pck_amt,set_type,trajlen,merc_seq,weekday,time_index_list
0,5240,2025-05-12,"[2025-05-12T11:23:06.000000000, 2025-05-12T11:...","[[-73.8074343, 40.909735], [-73.9277252, 40.81...",Trader Joe's,1,1478.72,train,30,"[[-8216206.003033994, 4999036.699901179], [-82...",0,"[68, 69, 71, 75, 76, 78, 79, 83, 89, 91, 98, 9..."
1,34299,2024-12-06,"[2024-12-06T00:01:53.000000000, 2024-12-06T00:...","[[-74.0592517, 40.7562114], [-74.0598949, 40.7...",u s postal service,0,3026.97,train,64,"[[-8244238.187774881, 4976449.130531165], [-82...",4,"[0, 1, 2, 4, 5, 7, 8, 9, 11, 12, 14, 15, 18, 2..."
2,34299,2025-02-15,"[2025-02-15T00:00:06.000000000, 2025-02-15T00:...","[[-74.0596466, 40.7565547], [-74.0592928, 40.7...",u s postal service,0,4129.42,train,70,"[[-8244282.147841794, 4976499.581186923], [-82...",5,"[0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 14, 16, 1..."
3,35404,2024-11-08,"[2024-11-08T00:34:19.000000000, 2024-11-08T06:...","[[-73.9452781, 40.7803969], [-73.9452432, 40.7...",Con edison,0,0.00,train,29,"[[-8231550.704659003, 4980004.019033388], [-82...",4,"[3, 36, 38, 39, 40, 41, 42, 44, 45, 46, 48, 51..."
4,54320,2025-05-06,"[2025-05-06T02:00:27.000000000, 2025-05-06T07:...","[[-73.9059497, 40.8341471], [-73.9078743, 40.8...",Sterling Nat Bnk,0,642.83,train,28,"[[-8227172.6871972885, 4987909.091900762], [-8...",1,"[12, 47, 48, 50, 52, 54, 55, 56, 57, 59, 60, 6..."


In [ ]:
# import pandas as pd
# test_df = pd.read_parquet("/home/sagemaker-user/TrajCL/data/parquet_files/test/nyc_df_v3_with_time/traj_test_df_v3_with_ts.parquet")

In [ ]:
# userids = test_df['userid'].unique()

In [7]:
import torch
device = torch.device("cuda:0")
checkpoint_file = "/home/sagemaker-user/TrajCL/exp/generic_v2_modified/generic_v2_TrajCL_best.pt"
checkpoint = torch.load(checkpoint_file)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

/tmp/ipykernel_7948/2271036465.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_file)


TrajCL(
  (clmodel): MoCo(
    (criterion): CrossEntropyLoss()
    (encoder_q): DualSTB(
      (pos_encoder): PositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (structural_attn): TransformerEncoder(
        (layers): ModuleList(
          (0-1): 2 x TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
            )
            (linear1): Linear(in_features=512, out_features=2048, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=2048, out_features=512, bias=True)
            (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (spatial_attn): SpatialMSM(


In [ ]:
from utils.traj import *
import pickle

from torch.nn.utils.rnn import pad_sequence
embs = pickle.load(open("/home/sagemaker-user/TrajCL/data/nyc_cell250_embdim256_embs.pkl", 'rb')).to('cpu').detach() # tensor
cellspace = pickle.load(open("/home/sagemaker-user/TrajCL/data/nyc_cell250_cellspace.pkl", 'rb'))

max_batch_size = 512
def infer_batch(traj, time_indices):
    traj_cell, traj_p = zip(*[merc2cell2(t, cellspace) for t in traj])
    traj_emb_p = [torch.tensor(generate_spatial_features(t, cellspace)) for t in traj_p]
    traj_emb_p = pad_sequence(traj_emb_p, batch_first = False).to(device)
    traj_emb_cell = [embs[list(t)] for t in traj_cell]
    traj_emb_cell = pad_sequence(traj_emb_cell, batch_first = False).to(device)
    traj_len = torch.tensor(list(map(len, traj_cell)), dtype = torch.long, device = device)
    time_indices = pad_sequence([torch.tensor(t, dtype=torch.long) for t in time_indices], batch_first=False, padding_value=-1).to(Config.device)
    # print(traj_emb_cell, traj_emb_p, traj_len)
    traj_embs = model.interpret(traj_emb_cell.float(), traj_emb_p.float(), traj_len, time_indices)
    return traj_embs

def infer(traj, time_indices):
    if len(traj)> max_batch_size:
        traj_embs = []
        for i in range(0, len(traj), max_batch_size):
            traj_batch = traj[i:i+max_batch_size]
            time_indices_batch = time_indices[i:i+max_batch_size]
            traj_embs.append(infer_batch(traj_batch, time_indices_batch))
        return torch.cat(traj_embs, dim=0)
    else:
        return infer_batch(traj, time_indices)



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
df_list[0]['time_index_list'].values[0]

In [ ]:
unique_userids = pd.concat([df['userid'] for df in df_list]).unique()
print(f"Total unique userids: {len(unique_userids)}")

unique_employernames = pd.concat([df['employername'] for df in df_list]).unique()
print(f"Total unique employernames: {len(unique_employernames)}")

In [ ]:
def get_data_for_userid(userid):
    df_user = pd.concat([df[df['userid']==userid] for df in df_list]).reset_index(drop=True)
    return df_user

def get_data_for_employername(df_user, employername):
    df_emp = df_user[df_user['employername']==employername].reset_index(drop=True)
    return df_emp


def get_data_for_partition(df_emp, partition):
    df_part = df_emp[df_emp['partition_id']==partition].reset_index(drop=True)
    df_part = df_part[~df_part['weekday'].isin([5, 6])].reset_index(drop=True) # filter out weekends
    df_part = df_part[df_part['pck_amt']>0].reset_index(drop=True) # filter out zero paycheck amount
    return df_part


def get_traj_and_time_data(df_part):
    traj = df_part['merc_seq'].values
    time_indices = df_part['time_index_list'].values
    return traj, time_indices

In [ ]:
def get_min_max_xy(traj):
    min_x = min([p[0] for p in traj])
    max_x = max([p[0] for p in traj])
    min_y = min([p[1] for p in traj])
    max_y = max([p[1] for p in traj])
    return min_x, max_x, min_y, max_y

def calculate_dx_dy(k, target_min_x, target_min_y, min_x, max_x, min_y, max_y):
    dx = target_min_x+ (k - max_x - min_x) / 2
    dy = target_min_y+ (k - max_y - min_y) / 2
    return dx, dy

def transform_traj(traj, dx, dy):
    new_traj = [[p[0] + dx, p[1] + dy] for p in traj]
    return new_traj

def normalize(trajs, cellspace):
    # trajs: list of [[lon, lat], [,], ...]

    # 1. augment the input traj in order to form 2 augmented traj views
    # 2. convert augmented trajs to the trajs based on mercator space by cells
    # 3. read cell embeddings and form batch tensors (sort, pad)

    trajs1, trajs2 = [], []
    time_indices1, time_indices2 = [], []
    final_min_x = 1000000000
    final_min_y = 1000000000
    final_max_x = -1000000000
    final_max_y = -1000000000
    max_len_meters = min(cellspace.x_max-cellspace.x_min, cellspace.y_max-cellspace.y_min)
    indices_to_omit = []
    for i, l in enumerate(trajs):
        min_x, max_x, min_y, max_y = get_min_max_xy(l)
        # if final_max_x - min_x> max_len_meters or final_max_y-min_y> max_len_meters or max_x - final_min_x> max_len_meters or max_y - final_min_y> max_len_meters or max_x-min_x> max_len_meters or max_y-min_y> max_len_meters:
        #     indices_to_omit.append(i)
        #     continue
        final_min_x = min(final_min_x, min_x)
        final_min_y = min(final_min_y, min_y)
        final_max_x = max(final_max_x, max_x)
        final_max_y = max(final_max_y, max_y)
    
    if max(final_max_x-final_min_x, final_max_y-final_min_y)>max_len_meters:
        # print(max(final_max_x-final_min_x, final_max_y-final_min_y), max_len_meters)
        return None
    dx, dy = calculate_dx_dy(max_len_meters, cellspace.x_min, cellspace.y_min, final_min_x, final_max_x, final_min_y, final_max_y)
    # print(final_min_x, final_max_x, final_min_y, final_max_y)
    # print(dx, dy)
    for i,l in enumerate(trajs):
        if i in indices_to_omit:
            continue
        new_l = transform_traj(l, dx, dy)
        trajs1.append(new_l)
    return trajs1

In [ ]:
def apply_dbscan(embs, target_min_similarity=0.85):
    from sklearn.cluster import DBSCAN     # require >= 0.9 cosine similarity
    eps = 1.0 - target_min_similarity    # cosine distance threshold
    n_embs = embs.shape[0]
    db = DBSCAN(eps=eps, min_samples=max(int(n_embs*0.3), 5), metric="cosine", n_jobs=-1).fit(embs)
    return db

In [ ]:
from tqdm import tqdm
model.eval()
output_dict = {}

# 0: moving, 1: static, 2: not enough data
def get_gt_and_pred_label(userid):
    user_data = get_data_for_userid(userid)
    employers = user_data['employername'].unique()
    for employer in employers:
        emp_data = get_data_for_employername(user_data, employer)
        partitions = emp_data['partition_id'].unique()
        for partition in partitions:
            # print(userid, employer, partition)
            part_data = get_data_for_partition(emp_data, partition)
            if len(part_data)<15:
                output_dict[(userid, employer, partition)] = 2
            else:
                                traj = get_traj_and_time_data(part_data)
                traj = normalize(traj, cellspace)
                # print(traj)
                if traj==None:
                    count+=1
                    pred_label = 3
                else:
                    embs = infer(traj).detach().cpu().numpy()
                    db_scan = apply_dbscan(embs, target_min_similarity=0.85)
                    labels = db_scan.labels_                      # shape: (n_samples,)
                    cluster_ids = [c for c in np.unique(labels) if c != -1]
                    if len(cluster_ids)>0:
                        pred_label = 1
                    else:
                        pred_label = 0
                output_dict[(userid, employer, partition)] = pred_label

    
    # if sum(test_data['paycheck_amount'].values) > 0:
    #     gt_label = 1
    # else:
    #     gt_label = 0
    # pred_label = 0
    # for i in range(len(test_embs)):
    #     test_vector = test_embs[i].unsqueeze(0)
    #     similarity = cosine_similarity(test_vector.numpy(), train_embs)[0]
    #     top_3_indices = np.argsort(similarity)[-3:][::-1]
    #     # print(i, top_3_indices)
    #     similarity = similarity[top_3_indices]
    #     # print(f"User: {userid}, Test Trajectory {test_data['traj_id'].values[i]}, Top 3 Train Trajectories: {train_data['traj_id'].values[top_3_indices]}, similarity: {similarity}, PCK Amount: {train_data['paycheck_amount'].values[top_3_indices]}")
    #     for sim, idx in zip(similarity, top_3_indices):
    #         if sim>0.85 and train_data['paycheck_amount'].values[idx]>0:
    #             pred_label = 1
    #             break
    # return gt_label, pred_label

for userid in tqdm(unique_userids):
    get_gt_and_pred_label(userid)


        


In [ ]:
# save output_dict
pickle.dump(output_dict, open("/home/sagemaker-user/TrajCL/exp/v2.1/nyc_test_hyp1_output_dict.pkl", 'wb'))

In [ ]:
output_dict

In [ ]:
variable_keys = []

In [ ]:
def get_data_userid_employer_partition(userid, employer, partition):
    df_user = pd.concat([df[df['userid']==userid] for df in df_list]).reset_index(drop=True)
    df_emp = df_user[df_user['employername']==employer].reset_index(drop=True)
    df_part = df_emp[df_emp['partition_id']==partition].reset_index(drop=True)
    df_part_fil = df_part[~df_part['weekday'].isin([5, 6])].reset_index(drop=True) # filter out weekends
    df_part_fil = df_part_fil[df_part_fil['pck_amt']>0].reset_index(drop=True) # filter out zero paycheck amount
    return df_part_fil

In [ ]:
df_part = get_data_userid_employer_partition(34299, 'u s postal service', 0)
df_part.sort_values('traj_date')